In [1]:
import cv2
import pytesseract
import numpy as np
import os, sys


In [2]:
os.getcwd()

'/home/odemakinde/Desktop/Image to Excel/Image-to-Excel/scripts'

In [3]:
os.chdir('/home/odemakinde/Desktop/Image to Excel/Image-to-Excel/test images/')

In [4]:
img = cv2.imread('menu2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
gray = cv2.bitwise_not(img_bin)
#cv2.imshow('image', gray)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [5]:
kernel = np.ones((2, 1), np.uint8)
img = cv2.erode(gray, kernel, iterations=1)
img = cv2.dilate(img, kernel, iterations=1)
out_below = pytesseract.image_to_string(img)
print("OUTPUT:", out_below)

OUTPUT:  

*FRESH GUACAMOLE *CEVICHE TROPICAL

“¥APPETIZERS S@

__ © Ceviche Tropical _ Cheese Sticks
TN LOIS Cele COAL eS OR LU Six cheese sticks, served with marinara dipping sauce 5.50
pico de gallo and olive oil 8.00 fe Tree
amarones Gratinados
_Fresh Guacamole Grilled jumbo shrimp chopped with onions, covered
Made right at your table..!! Mouthwatering fresh with melted Mexican cheese 7.99
chunks of avocado, diced onions, tomatoes, cilantro ;
Teo SOA ari Pollo Gratinado
p rilled chicken chopped with onions covered
Queso Fundido with melted Mexican cheese 6.49
Chorizo (Mexican sausage) with melted cheese 5.99 NE P|
itas de rollo
Tossed Salad Seven chicken wings bone in, served with celery
Lettuce, tomatoes, onions, bell pepper, cucumber, sticks and dipping sauce 6.99
mushroom and cheese. Served with your choice of Flavors:
dressing 3.75 Original, Teriyaki, BBO, Sweet & Sour, Mango Habanero

NACHOS DIPS

Cheese 3.99 Beef 5.00

Cheese Nachos aaele Bean 4.25 Spinach 4.00
Bean and Chees

In [6]:
splitByNewLines = out_below.split('\n')

KeyCategory = {}
count = 0
for k in splitByNewLines:
    if k.isupper():
        KeyCategory[k] = count
        count += 1
    else:
        count += 1
KeyCategory

{'*FRESH GUACAMOLE *CEVICHE TROPICAL': 2,
 '“¥APPETIZERS S@': 4,
 'NACHOS DIPS': 23,
 '*CHICKEN SOUP': 50,
 'VR RR RR RN': 57}

In [33]:
splitByNewLines[:5]

[' ', '', '*FRESH GUACAMOLE *CEVICHE TROPICAL', '', '“¥APPETIZERS S@']

In [22]:
list(KeyCategory.keys())[0]

'*FRESH GUACAMOLE *CEVICHE TROPICAL'

In [36]:
def createDummyCategory(Categories, gauge):
    DummyCategory = []
    start, stop = 0, 0
    try:
        for k in Categories.values():
            start = k
            mm = list(KeyCategory.values()).index(k)+1
            stop = list(KeyCategory.values())[mm]
            for f in range(start, stop):
                DummyCategory.append(list(KeyCategory.keys())[mm-1])
    except IndexError:
        start = k
        mm = list(KeyCategory.values()).index(k)+1
        stop = gauge
        for f in range(start, stop):
            DummyCategory.append(list(KeyCategory.keys())[mm-1])
        
    return DummyCategory
gauge = len(splitByNewLines)
dummyCategory = createDummyCategory(KeyCategory, gauge)

In [39]:
def CutData(data, data_keys):
    first = list(data_keys.keys())[0]
    index= data.index(first)
    
    return data[index:]

InputData = CutData(splitByNewLines, KeyCategory)

In [42]:
def confirmDimension(a,b):
    if len(a) == len(b):
        return True
    else:
        return False
    
out = confirmDimension(InputData, dummyCategory)
print(out)

True


In [52]:
InputData[4:8]

['__ © Ceviche Tropical _ Cheese Sticks',
 'TN LOIS Cele COAL eS OR LU Six cheese sticks, served with marinara dipping sauce 5.50',
 'pico de gallo and olive oil 8.00 fe Tree',
 'amarones Gratinados']

In [86]:
u = InputData[6].split(' ')
count = 0
for ba in u:
    try:
        print(float(ba))
        count += 1
    except ValueError:
        pass
    
print(u)
print(count)

8.0
['pico', 'de', 'gallo', 'and', 'olive', 'oil', '8.00', 'fe', 'Tree']
1


In [113]:
def CheckIntNum(data):
    indexes = []
    CountTrue = 0
    dataSplit = data.split(' ')
    for j in dataSplit:
        try:
            jint = float(j)
            CountTrue+= 1
            indexes.append(dataSplit.index(j))
        except ValueError:
            pass
    return CountTrue, indexes



In [114]:
def concatName(data, start, stop):
    #start and stop are both starting and ending indexes
    outString = ''
    for f in data[start:stop]:
        outString = outString + f + ' '
        
    return outString
name = ['eliael', 'elaiel', 'leielwiel', '45.2', 'wleile', 'esleiwl']
print(concatName(name, 0, 3))

eliael elaiel leielwiel 


In [119]:
Categories = {}#Name as key, dummyCategory and price as a tuple
for i, j in zip(InputData, dummyCategory):
    dataSplit = i.split(' ')
    if CheckIntNum(i)[0] >0:
        valOne, valTwo = CheckIntNum(i)
        start = 0
        for th in range(valOne):
            stop = valTwo[th]
            Name = concatName(dataSplit, start, stop)
            price = dataSplit[stop]
            Categories[Name] = (j, price)
            start = stop
        #except IndexError:
        #    pass
#     else:
#         pass
        

In [127]:
import pandas as pd
def FrameData(cate):
    keys, Name, price = [], [], []
    for k in cate.keys():
        keys.append(k)
        Name.append(cate[k][0])
        price.append(cate[k][1])
        
    DictFrame = {'Categories':Name, 'Name/Description':keys, 'price':price}
    frame = pd.DataFrame(DictFrame)
    return frame
    
frame = FrameData(Categories)
print(frame)
frame.to_csv('output.csv', index = False)


         Categories                                   Name/Description price
0   “¥APPETIZERS S@  TN LOIS Cele COAL eS OR LU Six cheese sticks, ...  5.50
1   “¥APPETIZERS S@                       pico de gallo and olive oil   8.00
2   “¥APPETIZERS S@  Made right at your table..!! Mouthwatering fre...  7.99
3   “¥APPETIZERS S@          Queso Fundido with melted Mexican cheese   6.49
4   “¥APPETIZERS S@      Chorizo (Mexican sausage) with melted cheese   5.99
5   “¥APPETIZERS S@  Lettuce, tomatoes, onions, bell pepper, cucumb...  6.99
6   “¥APPETIZERS S@                                          dressing   3.75
7       NACHOS DIPS                                            Cheese   3.99
8       NACHOS DIPS                                         3.99 Beef   5.00
9       NACHOS DIPS                          Cheese Nachos aaele Bean   4.25
10      NACHOS DIPS                                      4.25 Spinach   4.00
11      NACHOS DIPS                                   Bean and Cheese   4.75

In [ ]:
count = 0
for i,j in len(splitByNewLines, dummyCate):
    count += 1

# Scrap web

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://www.just-eat.com/'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
text = soup.get_text()
text.split(' ')